DATA PREPARATION

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import cv2

BASE_DIR = Path.cwd()
DATASET_PATH = BASE_DIR / "dataset_filtered"
LABEL_MAP = BASE_DIR / "label_map.txt"
MODEL_NAME = BASE_DIR / "crop_disease_model.keras"
EPOCHS = 12

In [2]:
if not DATASET_PATH.exists():
    raise FileNotFoundError(
        f"Dataset folder not found at {DATASET_PATH}\n"
        "Please download the dataset and place it in the project root as 'dataset_filtered/'"
    )

In [3]:
train_datagen = ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [4]:
val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

In [5]:
train_data = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

Found 16254 images belonging to 13 classes.


In [6]:
val_data = val_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

Found 4058 images belonging to 13 classes.


In [7]:
NUM_CLASSES = train_data.num_classes
labels = train_data.class_indices
with open(LABEL_MAP, "w") as f:
    for label, idx in labels.items():
        f.write(f"{idx}:{label}\n")

print("✅ Label map saved to", LABEL_MAP)

✅ Label map saved to d:\Lohith\BTECH CODES\Machine_Learning\Project\Crop-Disease\label_map.txt


MODEL ARCHITECTURE AND TRAINING

In [8]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False 

In [9]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
])

In [10]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 13)             │         3,341 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,589,261 (9.88 MB)

 Trainable params: 331,277 (1.26 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [12]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_data.classes),
    y=train_data.classes
)

class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

Class Weights: {0: np.float64(1.5628846153846154), 1: np.float64(1.5628846153846154), 2: np.float64(10.2484237074401), 3: np.float64(0.7346108650456477), 4: np.float64(1.5628846153846154), 5: np.float64(0.8182641965364479), 6: np.float64(1.6408237431859478), 7: np.float64(0.8823625210357744), 8: np.float64(0.9323696437790397), 9: np.float64(1.1123733917328222), 10: np.float64(0.29171901360422126), 11: np.float64(4.181631077952148), 12: np.float64(0.9821741494954378)}


In [13]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(MODEL_NAME, save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.3)
]

In [14]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=callbacks
)
print("Final Training Accuracy:", history.history["accuracy"][-1])

Epoch 1/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 353s 689ms/step - accuracy: 0.6721 - loss: 1.0749 - val_accuracy: 0.8418 - val_loss: 0.5188 - learning_rate: 3.0000e-04
Epoch 2/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 195s 384ms/step - accuracy: 0.8196 - loss: 0.5672 - val_accuracy: 0.8659 - val_loss: 0.4142 - learning_rate: 3.0000e-04
Epoch 3/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 190s 375ms/step - accuracy: 0.8522 - loss: 0.4656 - val_accuracy: 0.8667 - val_loss: 0.3847 - learning_rate: 3.0000e-04
Epoch 4/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 195s 385ms/step - accuracy: 0.8666 - loss: 0.4127 - val_accuracy: 0.8891 - val_loss: 0.3305 - learning_rate: 3.0000e-04
Epoch 5/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 200s 393ms/step - accuracy: 0.8794 - loss: 0.3702 - val_accuracy: 0.8783 - val_loss: 0.3492 - learning_rate: 3.0000e-04
Epoch 6/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 194s 382ms/step - accuracy: 0.8858 - loss: 0.3491 - val_accuracy: 0.8928 - val_loss: 0.3064 - learning_rate: 3.0000e-04
Epoch 7/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 

In [15]:
model.save(MODEL_NAME)
print("✅ Final model saved")


✅ Final model saved


In [16]:
label_map = {}
with open(LABEL_MAP, "r") as f:
    for line in f:
        idx, label = line.strip().split(":")
        label_map[int(idx)] = label

In [17]:
REMEDIATION_DB = {
    "Potato___Early_blight": {
        "organic": {
            "urgency": "Monitor & apply organic remedies.",
            "steps": [
                "Remove infected leaves",
                "Spray neem oil",
                "Improve air circulation"
            ],
            "dosage": "3 ml/L",
            "frequency": "Once every 7 days",
            "max_limit": "Do not exceed 5 ml/L",
            "base_cost": (100, 250)   # 🔥 NEW
        },
        "chemical": {
            "urgency": "Apply fungicide immediately.",
            "steps": [
                "Spray Mancozeb",
                "Repeat after 7 days",
                "Avoid irrigation for 24 hours"
            ],
            "dosage": "2 g/L",
            "frequency": "Once every 7 days",
            "max_limit": "Do not exceed 3 g/L",
            "base_cost": (250, 600)   # 🔥 NEW
        },
        "prevention": [
            "Avoid overhead irrigation",
            "Use disease-resistant varieties",
            "Crop rotation"
        ],
        "safety": [
            "Wear gloves & mask",
            "Do not spray during noon",
            "Keep away from children"
        ]
    }
}


In [18]:
model = tf.keras.models.load_model(MODEL_NAME)

dummy_input = np.zeros((1, 224, 224, 3), dtype=np.float32)
model(dummy_input)  # 👈 IMPORTANT: call model directly, not predict()


<tf.Tensor: shape=(1, 13), dtype=float32, numpy=
array([[2.3558036e-06, 2.7145757e-07, 1.5208500e-06, 5.6277501e-04,
        1.3047788e-05, 9.9529147e-01, 2.1486147e-05, 2.9396375e-03,
        3.5104102e-05, 1.7340095e-05, 7.0359831e-04, 1.2451320e-05,
        3.9906017e-04]], dtype=float32)>

In [19]:
def is_blurry(img_path):
    img = cv2.imread(img_path)
    variance = cv2.Laplacian(img, cv2.CV_64F).var()
    threshold = 100.0  # You can adjust this threshold
    return variance < threshold

In [126]:
def generate_gradcam(model, img_path, layer_name=None):
    """Generate a Grad-CAM heatmap overlay for the predicted class.

    Fixes Keras graph issues by building the Grad-CAM model using the
    MobileNetV2 base model's input graph (not Sequential's symbolic tensors).
    """
    base_model = model.layers[0]  # MobileNetV2 (Functional)

    # Pick a sensible conv layer if not provided
    if layer_name is None:
        for layer in reversed(base_model.layers):
            if isinstance(layer, tf.keras.layers.Conv2D):
                layer_name = layer.name
                break
        if layer_name is None:
            # Fallback: any layer with 'conv' in its name
            for layer in reversed(base_model.layers):
                if "conv" in layer.name.lower():
                    layer_name = layer.name
                    break

    if layer_name is None:
        raise ValueError("No convolutional layer found in base model.")

    conv_layer = base_model.get_layer(layer_name)

    # Build a Grad-CAM graph rooted at base_model.input to avoid tensor mismatch
    x = base_model.output
    for head_layer in model.layers[1:]:
        x = head_layer(x)
    preds = x

    grad_model = tf.keras.models.Model(
        inputs=base_model.input,
        outputs=[conv_layer.output, preds],
    )

    # Load and preprocess image
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0).astype(np.float32)

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(pooled_grads * conv_outputs, axis=-1)

    # Convert to numpy once, then use numpy/cv2 safely
    heatmap = heatmap.numpy()
    heatmap = np.maximum(heatmap, 0)
    heatmap /= (np.max(heatmap) + 1e-8)

    heatmap = cv2.resize(heatmap, (224, 224))
    heatmap = np.uint8(255 * heatmap)

    original = cv2.imread(img_path)
    if original is None:
        raise FileNotFoundError(f"Could not read image from path: {img_path}")
    original = cv2.resize(original, (224, 224))

    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(original, 0.6, heatmap, 0.4, 0)

    output_path = "heatmap_output.jpg"
    cv2.imwrite(output_path, overlay)
    return output_path

In [139]:
def draw_real_lesion_boxes(img_path):
    original = cv2.imread(img_path)
    if original is None:
        raise FileNotFoundError("Image not found")

    hsv = cv2.cvtColor(original, cv2.COLOR_BGR2HSV)

    # Brown / yellow lesion color range
    lower = np.array([5, 40, 40])
    upper = np.array([35, 255, 255])

    mask = cv2.inRange(hsv, lower, upper)

    # Morph cleanup
    kernel = np.ones((7, 7), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    boxes = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 250:  # reject noise
            x, y, w, h = cv2.boundingRect(cnt)
            if w > 15 and h > 15:
                boxes.append((x, y, w, h))

    # Draw boxes
    for (x, y, w, h) in boxes:
        cv2.rectangle(original, (x, y), (x + w, y + h), (0, 0, 255), 2)

    output_path = "boxed_output.jpg"
    cv2.imwrite(output_path, original)

    return output_path, len(boxes)


In [140]:
def predict_disease(img_path, confidence_threshold=0.45):
    if is_blurry(img_path):
        return {
            "status": "error",
            "message": "Image is too blurry. Please retake photo.",
            "confidence": 0.0
        }
    
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)[0]
    top_idx = np.argmax(preds)
    confidence = float(preds[top_idx])
    disease_name = label_map[top_idx]

    # Epic 3: Unknown/Unclear Result Story
    if confidence < confidence_threshold:
        return {
            "status": "unknown",
            "disease": "Unknown/Unclear",
            "message": "Please consult a human expert or retake the photo.",
            "confidence": round(confidence * 100, 2)
        }
    
    # Epic 3: Healthy Plant Story
    # Check if the predicted label contains the word "healthy"
    is_healthy = "healthy" in disease_name.lower()

    return {
        "status": "success",
        "disease": disease_name,
        "is_healthy": is_healthy,
        "confidence": round(confidence * 100, 2)
    }

In [141]:
print("Final Training Accuracy:", history.history["accuracy"][-1])
print("Final Validation Accuracy:", history.history["val_accuracy"][-1])


Final Training Accuracy: 0.9134982228279114
Final Validation Accuracy: 0.9115327596664429


In [142]:
def predict_top_k(img_path, k=3, min_conf=0.05):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)[0]
    top_indices = preds.argsort()[-k:][::-1]

    results = []
    for idx in top_indices:
        conf = float(preds[idx])
        if conf >= min_conf:
            results.append({
                "disease": label_map[idx],
                "confidence": round(conf * 100, 2)
            })

    return results


In [143]:
def estimate_cost(base_cost, severity):
    low, high = base_cost

    multiplier = {
        "Low": 0.8,
        "Medium": 1.0,
        "High": 1.4
    }.get(severity, 1.0)

    est_low = int(low * multiplier)
    est_high = int(high * multiplier)

    if est_high < 300:
        tier = "₹ Low"
    elif est_high < 700:
        tier = "₹₹ Medium"
    else:
        tier = "₹₹₹ High"

    return {
        "range": f"₹{est_low} – ₹{est_high}",
        "tier": tier
    }


In [144]:
def estimate_severity_from_heatmap(heatmap_path):
    heatmap = cv2.imread(heatmap_path)
    heatmap_gray = cv2.cvtColor(heatmap, cv2.COLOR_BGR2GRAY)

    # Threshold to isolate "hot" infected zones
    _, binary_map = cv2.threshold(heatmap_gray, 200, 255, cv2.THRESH_BINARY)

    infected_area = np.sum(binary_map == 255)
    total_area = binary_map.shape[0] * binary_map.shape[1]

    ratio = infected_area / total_area

    if ratio < 0.10:
        severity = "Low"
    elif ratio < 0.40:
        severity = "Medium"
    else:
        severity = "High"

    return {
        "severity": severity,
        "infected_ratio": round(ratio * 100, 2)
    }


In [145]:
def adjust_dosage(base_dosage, severity):
    value, unit = base_dosage.split()
    value = float(value)

    multiplier = {
        "Low": 0.7,
        "Medium": 1.0,
        "High": 1.3
    }.get(severity, 1.0)

    adjusted = round(value * multiplier, 2)
    return f"{adjusted} {unit}"

def get_remediation_plan(disease_name, severity="Low", preference="organic"):
    data = REMEDIATION_DB.get(disease_name)
    if not data:
        return None

    treatment = data.get(preference, data["organic"])

    adjusted_dosage = adjust_dosage(treatment["dosage"], severity)
    cost_info = estimate_cost(treatment["base_cost"], severity)

    escalate = severity == "High"

    return {
        "mode": preference,
        "urgency": treatment["urgency"],
        "treatment_steps": treatment["steps"],
        "dosage": adjusted_dosage,
        "application_frequency": treatment["frequency"],
        "max_limit": treatment["max_limit"],
        "cost_range": cost_info["range"],
        "cost_tier": cost_info["tier"],
        "prevention_tips": data["prevention"],
        "safety_warnings": data["safety"],
        "escalate_to_expert": escalate
    }


In [150]:
def full_predict_with_heatmap(img_path, treatment_mode="organic"):
    result = predict_disease(img_path)

    if result["status"] != "success":
        return result

    if result.get("is_healthy"):
        result["message"] = "No treatment required. Your plant is healthy."
        result["remediation"] = None
        result["severity"] = "None"
        result["infected_ratio"] = 0.0
        return result

    heatmap_path = generate_gradcam(model, img_path)
    severity_info = estimate_severity_from_heatmap(heatmap_path)

    # 🔥 NEW: Draw bounding boxes
    boxed_image_path, spot_count = draw_real_lesion_boxes(img_path)

    remediation = get_remediation_plan(
        result["disease"],
        severity_info["severity"],
        treatment_mode
    )

    result["heatmap"] = heatmap_path
    result["boxed_image"] = boxed_image_path
    result["spot_count"] = spot_count
    result["severity"] = severity_info["severity"]
    result["infected_ratio"] = severity_info["infected_ratio"]
    result["remediation"] = remediation

    return result


In [151]:
def pretty_print_result(result):
    print("\n========== AI CROP DIAGNOSIS REPORT ==========\n")

    print(f"Status           : {result.get('status')}")
    print(f"Disease          : {result.get('disease')}")
    print(f"Healthy          : {result.get('is_healthy')}")
    print(f"Confidence       : {result.get('confidence')} %")
    print(f"Severity         : {result.get('severity')}")
    print(f"Infected Ratio   : {result.get('infected_ratio')} %")

    if "remediation" in result and result["remediation"] is not None:
        r = result["remediation"]   # 🔥 DEFINE FIRST

        print("\n--- Remediation Plan ---")
        print(f"Mode             : {r.get('mode')}")
        print(f"Urgency          : {r.get('urgency')}")
        print(f"Dosage           : {r.get('dosage')}")
        print(f"Frequency        : {r.get('application_frequency')}")
        print(f"Max Limit        : {r.get('max_limit')}")
        print(f"Cost Range       : {r.get('cost_range')}")
        print(f"Cost Tier        : {r.get('cost_tier')}")
        print(f"Infected Spots  : {result.get('spot_count')}")
        print(f"Boxed Image     : {result.get('boxed_image')}")


        print("\nTreatment Steps:")
        for i, step in enumerate(r.get("treatment_steps", []), 1):
            print(f"  {i}. {step}")

        print("\nPrevention Tips:")
        for tip in r.get("prevention_tips", []):
            print(f"  - {tip}")

        print("\nSafety Warnings:")
        for warn in r.get("safety_warnings", []):
            print(f"  ⚠ {warn}")

        print(f"\nEscalate to Expert : {'Yes' if r.get('escalate_to_expert') else 'No'}")

    else:
        print("\nNo treatment required.")

    print("\n=============================================\n")


In [152]:
if __name__ == "__main__":
    TEST_IMAGE = r'0a8a68ee-f587-4dea-beec-79d02e7d3fa4___RS_Early.B 8461.JPG'
    result = full_predict_with_heatmap(TEST_IMAGE, "organic")
    pretty_print_result(result)

    print("Boxed output saved as:", result["boxed_image"])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

========== AI CROP DIAGNOSIS REPORT ==========

Status           : success
Disease          : Potato___Early_blight
Healthy          : False
Confidence       : 100.0 %
Severity         : Low
Infected Ratio   : 1.24 %

--- Remediation Plan ---
Mode             : organic
Urgency          : Monitor & apply organic remedies.
Dosage           : 2.1 ml/L
Frequency        : Once every 7 days
Max Limit        : Do not exceed 5 ml/L
Cost Range       : ₹80 – ₹200
Cost Tier        : ₹ Low
Infected Spots  : 7
Boxed Image     : boxed_output.jpg

Treatment Steps:
  1. Remove infected leaves
  2. Spray neem oil
  3. Improve air circulation

Prevention Tips:
  - Avoid overhead irrigation
  - Use disease-resistant varieties
  - Crop rotation

Safety Warnings:
  ⚠ Wear gloves & mask
  ⚠ Do not spray during noon
  ⚠ Keep away from children

Escalate to Expert : No


Boxed output saved as: boxed_output.jpg


In [153]:
for sev in ["Low", "Medium", "High"]:
    print(f"\n--- {sev} Severity / Organic Mode ---")
    severity_override = {"Low": 0.05, "Medium": 0.25, "High": 0.55}[sev]

    result = full_predict_with_heatmap(TEST_IMAGE, "organic")
    result["severity"] = sev  # simulate severity
    result["remediation"] = get_remediation_plan(
        result["disease"], sev, "organic"
    )

    pretty_print_result(result)



--- Low Severity / Organic Mode ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

========== AI CROP DIAGNOSIS REPORT ==========

Status           : success
Disease          : Potato___Early_blight
Healthy          : False
Confidence       : 100.0 %
Severity         : Low
Infected Ratio   : 1.24 %

--- Remediation Plan ---
Mode             : organic
Urgency          : Monitor & apply organic remedies.
Dosage           : 2.1 ml/L
Frequency        : Once every 7 days
Max Limit        : Do not exceed 5 ml/L
Cost Range       : ₹80 – ₹200
Cost Tier        : ₹ Low
Infected Spots  : 7
Boxed Image     : boxed_output.jpg

Treatment Steps:
  1. Remove infected leaves
  2. Spray neem oil
  3. Improve air circulation

Prevention Tips:
  - Avoid overhead irrigation
  - Use disease-resistant varieties
  - Crop rotation

Safety Warnings:
  ⚠ Wear gloves & mask
  ⚠ Do not spray during noon
  ⚠ Keep away from children

Escalate to Expert : No



--- Medium Severity / Organic Mode ---
1/1 ━━━━━━━━━━━━━━━━━━━━